In [ ]:
import asyncio
import nest_asyncio
import Services.Receiver.TestWebsocketReceiver as futures_ws
import aiohttp
from SystemConfig import Streaming


nest_asyncio.apply()

symbols = Streaming.symbols
intervals = Streaming.intervals

q_ws_async = asyncio.Queue()

base_url="wss://stream.binance.com:9443/ws/"

class ws_mechine(futures_ws.WebsocketReceiver):
    def __init__(self, base_url:str, symbols:list, queue:asyncio.Queue, ):
        super().__init__(symbols = symbols, base_url = base_url)

        self.queue = queue
        self.intervals = None
        self.stream_type = None
        
    async def ws_setting(self, intervals):
        self.intervals = [f"interval_{i}" for i in intervals]
        self.stream_type = "kline"
        url = self._streams(self.intervals)
        async with aio
    def receiver_ws(self):
        

In [3]:
ins_dummy = ws_mechine(base_url, symbols, q_ws_async)

In [4]:
ins_dummy.kline_args_update(['1m','3m'])

In [5]:
ins_dummy.__dict__

{'BASE_URL': 'wss://stream.binance.com:9443/ws/',
 'symbols': ['BTCUSDT', 'TRXUSDT', 'ETHUSDT', 'XRPUSDT', 'SOLUSDT', 'BNBUSDT'],
 'stream_type': 'kline',
 'intervals': ['interval_1m', 'interval_3m'],
 'ENDPOINT': ['ticker',
  'trade',
  'miniTicker',
  'kline_1m',
  'kline_3m',
  'kline_5m',
  'kline_15m',
  'kline_30m',
  'kline_1h',
  'kline_2h',
  'kline_4h',
  'kline_6h',
  'kline_8h',
  'kline_12h',
  'kline_1d',
  'kline_3d',
  'kline_1w',
  'kline_1M',
  'depth',
  '24hrTicker',
  'aggTrade'],
 'queue': <Queue at 0x10e6e2f60 maxsize=0>}